In [3]:
# Import Libs
import time
init_time = time.time()

import os
import pickle
import numpy as np
import ROOT

from functions import HistogramFunctions,AuxiliarFunctions,FigureFunctions,FitFunctions,rootnotes

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['lines.linewidth'] = 2
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['ytick.labelsize'] = 20

current_time = time.time()
print 'Time to import libraries: %1.3f seconds'%(current_time-init_time)


Time to import libraries: 4.240 seconds


In [2]:
# Read Data

from keras.utils import np_utils

init_time = time.time()

from sklearn import datasets

processes = "lvbb125"
output = "/home/isabella/Workspace/python/PtRelAnalysis/PtRelAnalysis/Results/DataFiles"

# import some data to play with

signal = np.load(output+"/"+"lvbb125Array.npy") 
trgt = data.target # lvbb data
data_colors = ['b','r']
data_labels = ['signal','background']

# for classification -> target max sparse
trgt_sparse = np_utils.to_categorical(trgt)

current_time = time.time()
print 'Time to import data: %1.3f seconds'%(current_time-init_time)

Using Theano backend.


AttributeError: 'module' object has no attribute 'load_is'